In [13]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv

# 1. 환경변수 에서 api키 로딩
load_dotenv()
SERVICE_KEY = os.getenv("OPENWEATHER_API_KEY")

# 2. 설정값
CSV_FILE = "short_forecast.csv"
NX, NY = 60, 127  # 서울 (격자 위치)
API_URL = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"

# 3. 기존 단기예보불러오기
def load_existing_forecast():
    if os.path.exists(CSV_FILE):
        return pd.read_csv(CSV_FILE)
    return pd.DataFrame()

# 4. 단기예보 수집 (하루 전 base_time 계산 포함)
def get_yesterday_base_time():
    base_times = ['2300', '2000', '1700', '1400', '1100', '0800', '0500', '0200']
    base_time = base_times[0]
    base_date = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
    return base_date, base_time

def fetch_short_forecast():
    base_date, base_time = get_yesterday_base_time()

    params = {
        "serviceKey": SERVICE_KEY,
        "numOfRows": "1000",
        "pageNo": "1",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": NX,
        "ny": NY
    }

    print("📡 단기예보 수집 시작")
    try:
        response = requests.get(API_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        items = data.get("response", {}).get("body", {}).get("items", {}).get("item", [])
        print(f"✅ {len(items)}건 수집 완료")
        return items
    except Exception as e:
        print(f"❌ 수집 실패: {e}")
        return []

# 5. 새로운 예보만 저장
def save_incremental_forecast(items):
    if not items:
        print("ℹ️ 저장할 데이터가 없습니다.")
        return

    df_new = pd.DataFrame(items)
    df_old = load_existing_forecast()

    if not df_old.empty:
        merge_keys = ["fcstDate", "fcstTime", "category", "nx", "ny"]
        df_merge = pd.merge(df_new, df_old, on=merge_keys, how="left", indicator=True)
        df_filtered = df_merge[df_merge["_merge"] == "left_only"][df_new.columns]
        print(f"🆕 새로운 예보 {len(df_filtered)}건 저장됨")
    else:
        df_filtered = df_new
        print(f"📁 첫 저장: {len(df_filtered)}건")

    write_header = not os.path.exists(CSV_FILE)
    df_filtered.to_csv(CSV_FILE, mode='a', index=False, header=write_header)
    print(f"📅 {datetime.now().strftime('%Y-%m-%d')} 저장 완료")

# 실행
if __name__ == "__main__":
    items = fetch_short_forecast()
    save_incremental_forecast(items)


📡 단기예보 수집 시작
❌ 수집 실패: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=prE5nprnhB9u6l%2BsEZ7eyTE1s3G8HsLeMqpdV9c4RhUAGp03obu1bxAzYTcnbg4x2%2B4y8NbOTC0Yw5S0Y4hGIA%3D%3D&numOfRows=1000&pageNo=1&dataType=JSON&base_date=20250713&base_time=2300&nx=60&ny=127 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1017)')))
ℹ️ 저장할 데이터가 없습니다.
